In [35]:
import os
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/mohit1d.lp/E2EDSProject1.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "mohit1d.lp"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "9373733327493dd1ae55bf9e767a047779257c65"

In [36]:
os.chdir(r"C:\Users\Mohit\Desktop\Mohit\Projects\E2EDSProject1")
os.getcwd()

'C:\\Users\\Mohit\\Desktop\\Mohit\\Projects\\E2EDSProject1'

In [37]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metric_file_name: Path
    all_params: dict
    target_column: str
    mlflow_uri: str

In [38]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories, save_json
from src.datascience.entity.config_entity import (DataIngestionconfig, DataValidationConfig, DataTransformationConfig, ModelTrainerConfig)

class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionconfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionconfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config
    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir=config.unzip_data_dir,
            all_schema=schema
        )

        return data_validation_config

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path
        )

        return data_transformation_config
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            target_column=schema.name

        )

        return model_trainer_config

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            metric_file_name=config.metric_file_name,
            all_params=params,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/mohit1d.lp/E2EDSProject1.mlflow"
        )

        return model_evaluation_config


In [39]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [40]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)

        return  rmse, mae, r2
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        X_test = test_data.drop(columns=self.config.target_column)
        y_test = test_data[self.config.target_column]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_uri_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        mlflow.create_experiment("winequality-red")


        with mlflow.start_run():
            predicted_qualities = model.predict(X_test)

            (rmse, mae, r2) = self.eval_metrics(y_test, predicted_qualities)

            # Saving metrics in local as json
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            # Logging params and metrics in MLFLOW
            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)

            # Model registry does not work with file store
            if tracking_uri_type_store != "file":
                
                # Register the model
                # There are other ways to use the Model Registry, depending on the use case.
                # Please refer to the documentation for more info
                # MLFLOW Model Registry -> Type this in Google
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticNetModel")
            else:
                mlflow.sklearn.log_model("model")

In [41]:
print(mlflow.get_tracking_uri())


https://dagshub.com/mohit1d.lp/E2EDSProject1.mlflow


In [42]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()

except Exception as e:
    raise e



[2024-12-28 22:18:34,522: INFO: common: yaml file: config\config.yaml loaded sucessfully]
[2024-12-28 22:18:34,524: INFO: common: yaml file: params.yaml loaded sucessfully]
[2024-12-28 22:18:34,526: INFO: common: yaml file: schema.yaml loaded sucessfully]
[2024-12-28 22:18:34,527: INFO: common: Created directory at: artifacts]
[2024-12-28 22:18:34,528: INFO: common: Created directory at: artifacts/model_evaluation]
[2024-12-28 22:18:35,338: INFO: common: Json file saved at: artifacts\model_evaluation\metrics.json]


2024/12/28 22:18:43 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'ElasticNetModel'.
2024/12/28 22:18:47 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticNetModel, version 1
Created version '1' of model 'ElasticNetModel'.


🏃 View run aged-lamb-490 at: https://dagshub.com/mohit1d.lp/E2EDSProject1.mlflow/#/experiments/0/runs/3cc496c631c1400f94067f80b11be141
🧪 View experiment at: https://dagshub.com/mohit1d.lp/E2EDSProject1.mlflow/#/experiments/0
